In [ ]:
!pip install builtwith

  Preparing metadata (setup.py) ... done
  Created wheel for builtwith: filename=builtwith-1.3.4-py3-none-any.whl size=36077 sha256=e122632e1036a4595bfa74bdfa231fdfd0170c8f55c13521e114bda9b88d6eaf
  Stored in directory: /root/.cache/pip/wheels/7f/2d/b2/606e3df914d4aeeab99c4a4e3e9a61673d2293c2e346db00c8
Successfully built builtwith


In [ ]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://pta.trunojoyo.ac.id')
print(res)

{'web-servers': ['Nginx'], 'javascript-frameworks': ['jQuery', 'jQuery UI']}


# CRAWLING PTA

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def pta_fakultas_teknik_limit5():
    data = {
        "penulis": [],
        "judul": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "abstrak": [],
        "prodi": []
    }

    # Daftar prodi Fakultas Teknik
    prodi_fatek = {
        10: "Teknik Informatika",
        11: "Teknik Industri",
        12: "Teknik Mesin",
        13: "Teknik Elektro",
        14: "Teknik Mekatronika",
        15: "Sistem Informasi",
        16: "Teknik Multimedia dan Jaringan",
        17: "Manajemen Informatika",
        18: "Mekatronika"
    }

    for prodi_id, prodi_name in prodi_fatek.items():
        for page in range(1, 6):  # cuma ambil halaman 1,2,3,4,5
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi_id}/{page}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            if not jurnals:
                break  # kalau sudah tidak ada data, berhenti

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']
                response = requests.get(link)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip()

                penulis_element = isi.select_one('span:contains("Penulis")')
                penulis_text = penulis_element.text if penulis_element else ""
                penulis_parts = penulis_text.split(' : ')
                penulis = penulis_parts[1].strip() if len(penulis_parts) > 1 else ""

                pembimbing_pertama_element = isi.select_one('span:contains("Dosen Pembimbing I")')
                pembimbing_pertama_text = pembimbing_pertama_element.text if pembimbing_pertama_element else ""
                pembimbing_pertama_parts = pembimbing_pertama_text.split(' : ')
                pembimbing_pertama = pembimbing_pertama_parts[1].strip() if len(pembimbing_pertama_parts) > 1 else ""

                pembimbing_kedua_element = isi.select_one('span:contains("Dosen Pembimbing II")')
                pembimbing_kedua_text = pembimbing_kedua_element.text if pembimbing_kedua_element else ""
                pembimbing_kedua_parts = pembimbing_kedua_text.split(' : ')
                pembimbing_kedua = pembimbing_kedua_parts[1].strip() if len(pembimbing_kedua_parts) > 1 else ""


                paragraf = isi.select('p[align="justify"]')
                abstrak = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else ""

                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["abstrak"].append(abstrak)
                data["prodi"].append(prodi_name)

            print(f"✅ {prodi_name} - Halaman {page} selesai.")

    df = pd.DataFrame(data)
    df.to_csv("pta_fatek_5halaman.csv", index=False, encoding="utf-8-sig")
    print(f"\n📊 Total entri: {len(df)}")
    print("✅ Data berhasil diambil dari 5 halaman tiap   prodi Fakultas Teknik.")
    return df

# Jalankan
pta_fakultas_teknik_limit5()

✅ Teknik Informatika - Halaman 1 selesai.
✅ Teknik Informatika - Halaman 2 selesai.
✅ Teknik Informatika - Halaman 3 selesai.
✅ Teknik Informatika - Halaman 4 selesai.
✅ Teknik Informatika - Halaman 5 selesai.
✅ Teknik Industri - Halaman 1 selesai.
✅ Teknik Industri - Halaman 2 selesai.
✅ Teknik Industri - Halaman 3 selesai.
✅ Teknik Industri - Halaman 4 selesai.
✅ Teknik Industri - Halaman 5 selesai.
✅ Teknik Mesin - Halaman 1 selesai.
✅ Teknik Mesin - Halaman 2 selesai.
✅ Teknik Mesin - Halaman 3 selesai.
✅ Teknik Mesin - Halaman 4 selesai.
✅ Teknik Mesin - Halaman 5 selesai.
✅ Teknik Elektro - Halaman 1 selesai.
✅ Teknik Elektro - Halaman 2 selesai.
✅ Teknik Elektro - Halaman 3 selesai.
✅ Teknik Elektro - Halaman 4 selesai.
✅ Teknik Elektro - Halaman 5 selesai.
✅ Teknik Mekatronika - Halaman 1 selesai.
✅ Teknik Mekatronika - Halaman 2 selesai.
✅ Teknik Mekatronika - Halaman 3 selesai.
✅ Teknik Mekatronika - Halaman 4 selesai.
✅ Teknik Mekatronika - Halaman 5 selesai.
✅ Sistem Inform

,penulis,judul,pembimbing_pertama,pembimbing_kedua,abstrak,prodi
0,A.Ubaidillah S.Kom,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Budi Setyono M.T,,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika
1,"M. Basith Ardianto,",APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"Drs. Budi Soesilo, MT",,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika
2,"Akhmad Suyandi, S.Kom",RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,"Drs. Budi Soesilo, M.T",,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika
3,Heri Supriyanto,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,"Mulaab, S.Si., M.Kom",,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika
4,Septian Rahman Hakim,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,"Arik Kurniawati, S.Kom., M.T.",,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika
...,...,...,...,...,...,...
220,Syahfitri Tika Suci,PENGARUH PEMBELAJARAN BERBASIS NILAI DENGAN ME...,"Ariesta Kartika Sari, S. Si., M. Pd",,Penelitian ini bertujuan untuk mengetahui peng...,Mekatronika
221,Putri Aprilia Prihatin,PERBANDINGAN MODEL PEMBELAJARAN KOOPERATIF TIP...,"Ariesta Kartika Sari, S. Si., M. Pd",,Sikap demokratis tidak hanya menuntut siswa ak...,Mekatronika
222,Tiara Ayu Rahma Illahi,PENGARUH PENGGUNAAN MEDIA CD INTERAKTIF TERHAD...,"Sulaiman, S.Pd., M.Pd",,Penelitian “Pengaruh Penggunaan Media CD Inter...,Mekatronika
223,Dianatur Rosyida,Pengaruh Penerapan Pendekatan Konstruktivistik...,"Hani'ah, S. Pd., M. Pd",,Pembelajaran bahasa Indonesia meliputi empat a...,Mekatronika


# LINK KELUAR PTA

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def crawl_links_teknik(pages):
    data = []
    no = 1

    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Ambil semua link
            links = soup.find_all('a', href=True)
            for link in links:
                href = link['href']
                teks = link.get_text(strip=True)

                # Filter hanya link keluar (mulai dengan http & bukan link internal teknik.trunojoyo.ac.id)
                if href.startswith("http") and "teknik.trunojoyo.ac.id" not in href:
                    data.append({
                        "No": no,
                        "Page": page,
                        "Link Keluar": href,
                        "Teks": teks if teks else "-"
                    })
                    no += 1

        except requests.exceptions.RequestException as e:
            print(f"Gagal mengakses {page}: {e}")

    # Buat DataFrame
    df = pd.DataFrame(data)

    print("\n🔗 Daftar Link Keluar Fakultas Teknik:")
    print(df.to_string(index=False))

    # Simpan hasil ke CSV
    df.to_csv("link_keluar_fakultas_teknik.csv", index=False, encoding="utf-8-sig")
    print("\n✅ Data link keluar berhasil disimpan ke link_keluar_fakultas_teknik.csv")

    return df


# Daftar halaman Fakultas Teknik yang ingin diambil link keluarnya
pages_to_crawl = [
    "https://teknik.trunojoyo.ac.id/",
    "https://teknik.trunojoyo.ac.id/sekilas/",
    "https://teknik.trunojoyo.ac.id/berita/",
    "https://teknik.trunojoyo.ac.id/prodi/",
]

# Jalankan
crawl_links_teknik(pages_to_crawl)

Gagal mengakses https://teknik.trunojoyo.ac.id/berita/: 404 Client Error: Not Found for url: https://teknik.trunojoyo.ac.id/berita/
Gagal mengakses https://teknik.trunojoyo.ac.id/prodi/: 404 Client Error: Not Found for url: https://teknik.trunojoyo.ac.id/prodi/

🔗 Daftar Link Keluar Fakultas Teknik:
 No                                    Page                                                                           Link Keluar                                    Teks
  1         https://teknik.trunojoyo.ac.id/                                               https://fttrunojoyo.perpustakaan.co.id/                      Ruang Baca Digital
  2         https://teknik.trunojoyo.ac.id/ https://drive.google.com/file/d/13vrMWMRLc51yaZPqJw_wBPAgRqiH0315/view?usp=drive_link                         Teknik Industri
  3         https://teknik.trunojoyo.ac.id/   https://drive.google.com/file/d/1p0IjDah2BT7MSg4DPSarLJhbYSVP8lOx/view?usp=drivesdk                            Teknik Mesin
  4         https:/

,No,Page,Link Keluar,Teks
0,1,https://teknik.trunojoyo.ac.id/,https://fttrunojoyo.perpustakaan.co.id/,Ruang Baca Digital
1,2,https://teknik.trunojoyo.ac.id/,https://drive.google.com/file/d/13vrMWMRLc51ya...,Teknik Industri
2,3,https://teknik.trunojoyo.ac.id/,https://drive.google.com/file/d/1p0IjDah2BT7MS...,Teknik Mesin
3,4,https://teknik.trunojoyo.ac.id/,https://drive.google.com/file/d/1xThmCp00FGmAW...,Teknik Informatika
4,5,https://teknik.trunojoyo.ac.id/,https://drive.google.com/file/d/1dv65FxClUVDwu...,Sistem Informasi
...,...,...,...,...
127,128,https://teknik.trunojoyo.ac.id/sekilas/,https://forms.gle/tiuNL46SEDSwisUx8,Survey Kepuasan Layanan FT
128,129,https://teknik.trunojoyo.ac.id/sekilas/,https://forms.gle/KhxRqjckV4tQ9UWRA,Survey Pemahaman Visi & Misi
129,130,https://teknik.trunojoyo.ac.id/sekilas/,https://drive.google.com/file/d/1WRVnrFu8sazFU...,Rencana Induk Penelitian
130,131,https://teknik.trunojoyo.ac.id/sekilas/,https://drive.google.com/file/d/13TbpU9hyXd-1G...,Rencana Induk Pengabdian


# Crawling Berita melalui website detik.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"

In [ ]:
#Fungsi untuk membersihkan konten artikel
def clean_content(content_element):
    if not content_element:
        return "Content Not Found"
    return " ".join(content_element.stripped_strings)

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # berhenti jika artikel sudah cukup

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = clean_content(content_element)

        judul.append(title)
        isi.append(content)
        kategori_list.append(kategori)

        # tampilkan 10 judul pertama saja sebagai indikator
        if len(judul) <= 100:
            print(title)

# URL kategori Detik
base_urls = [
    "https://news.detik.com/indeks",
    "https://sport.detik.com/indeks",
]

categories = [
    "Politik",
    "Olahraga",
]

# Inisialisasi list untuk menyimpan data
judul = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 100

# Looping setiap kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}?page={page}"   # ✅ perbaikan format URL
        get_data(url, category, min_articles_per_category)
        time.sleep(2)
        page += 1

# Buat dataframe
df = pd.DataFrame({"judul": judul, "isi": isi, "kategori": kategori_list})

# Simpan ke CSV
df.to_csv("CrawlBerita_Politik-Olahraga.csv", index=False, encoding="utf-8-sig")

print("✅ Crawling selesai, data disimpan ke CrawlBerita_Politik-Olahraga.csv")

Title Not Found
Mensos Ajak Kepala Daerah Kolaborasi Sukseskan Program Prioritas Prabowo
Mendagri & CIO Danantara Bahas Penguatan Pendidikan-Pengelolaan Sampah
Video: Alasan Tersangka Penculikan Kacab Bank Tak Dijerat Pasal Pembunuhan
Kemnaker Catat 16.900 Pengunjung PTSA, Mayoritas Urus K3 & WLKP
Potret Macet di Matraman, Motor Serobot Busway hingga Trotoar
Update Banjir di Jaksel: 6 RT Tergenang, Ketinggian Air Capai 50 Cm
HUT Ke-15 BNPP, Mendagri Bagi 2.000 Paket Sembako ke Warga Tanah Tinggi
Video: Polri Ungkap Hasil Visum Sementara Kacab Bank yang Tewas
Menteri Agus Apresiasi Program Bina Napi-Ketahanan Pangan di Jajaran Permasyarakatan Sulsel
Identifikasi Bahaya Laten Pembangunan yang Tidak Berkelanjutan
Yusril Koordinasi ke Polri soal 3 Orang Diduga Hilang Saat Demo Ricuh Jakarta
Banyak Motor Mogok gegara Terobos Banjir di Jalan Ciledug Raya Jaksel
Jalan Ciledug Raya Cipulir Banjir Diduga Dipicu Turap Jebol, Ini Titiknya
Menko Kumham-Imipas Audiensi dengan Koalisi Sipil Bahas Re

In [ ]:
df=pd.read_csv("CrawlBerita_Politik-Olahraga.csv")
df.head(200)

,judul,isi,kategori
0,Title Not Found,Content Not Found,Politik
1,Mensos Ajak Kepala Daerah Kolaborasi Sukseskan...,Jakarta - Menteri Sosial Saifullah Yusuf (Gus ...,Politik
2,Mendagri & CIO Danantara Bahas Penguatan Pendi...,Jakarta - Menteri Dalam Negeri Muhammad Tito K...,Politik
3,Video: Alasan Tersangka Penculikan Kacab Bank ...,Polisi mengungkap alasan para tersangka pencul...,Politik
4,"Kemnaker Catat 16.900 Pengunjung PTSA, Mayorit...",Jakarta - Kementerian Ketenagakerjaan (Kemnake...,Politik
...,...,...,...
195,Hasil Hong Kong Open 2025: Febriana/Meilysa Ka...,Jakarta - Pasangan ganda putri Indonesia Febri...,Olahraga
196,Hasil Hong Kong Open 2025: Apri/Fadia Dikalahk...,Daftar Isi Jadwal dan hasil wakil Indonesia di...,Olahraga
197,Jadwal Hong Kong Open 2025 Hari Ini: 8 Wakil R...,Hong Kong - Delapan wakil Indonesia bertanding...,Olahraga
198,Timnas Wushu Indonesia Bawa Enam Medali dari K...,Jakarta - Timnas Wushu Indonesia sudah menunta...,Olahraga
